In [ ]:
! pip install pymongo
! pip install python-dotenv

In [3]:
import pymongo
import json
from dotenv import dotenv_values

In [ ]:
env_name = "myconfig.env" 
config = dotenv_values(env_name)

# Connection string
mongo_conn = config['cosmos_connection_string']
mongo_client = pymongo.MongoClient(mongo_conn)

# Database name
DATABASE_NAME = "ProductRecommendation"
db = mongo_client[DATABASE_NAME]

# Drop the database if it already exists (consider if this is really needed, as it will remove all existing data)
# mongo_client.drop_database(DATABASE_NAME)

# Collection names
collection_names = ["ProductCollection", "ActualRating", "PredictedRating"]

# Iterate through the collection names and create them if they do not exist
for collection_name in collection_names:
    if collection_name not in db.list_collection_names():
        # Creates a collection
        db.create_collection(collection_name)
        print(f"Created collection '{collection_name}'.")
    else:
        print(f"Using existing collection: '{collection_name}'.")

In [5]:
# create vector index on ProductCollection
db.command({
  'createIndexes': 'ProductCollection',
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "Embedding": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-hnsw',
        'm': 16,
        'efConstruction': 40,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
});

In [6]:
# Index creation for ProductType and rated product ids 

collection_product = db['ProductCollection']
collection_product.create_index([("Type", 1), ("Id", 1)])

'Type_1_Id_1'

In [ ]:
# load the product catalog data
data_file = open(file="./data/catalog/catalog.json", mode="r") 
data = json.load(data_file)
data_file.close()

result = collection_product.insert_many(data)

print(f"Number of data points added: {len(result.inserted_ids)} in {collection_product.name}")